In [0]:

from pyspark.sql import SparkSession
from pyspark import SparkConf
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from datetime import datetime

In [0]:
import databricks.koalas as ks


In [0]:
filepath = "s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json"

In [0]:
df = spark.read.format('json').option('header', 'true').option('inferSchema', 'true').load(filepath)

In [0]:
df.show()

+--------------------+-------+
|                data|     id|
+--------------------+-------+
|{10, [Multi-playe...|     10|
|{1000000, [Single...|1000000|
|{1000010, [Single...|1000010|
|{1000030, [Multi-...|1000030|
|{1000040, [Single...|1000040|
|{1000080, [Multi-...|1000080|
|{1000100, [Single...|1000100|
|{1000110, [Multi-...|1000110|
|{1000130, [Single...|1000130|
|{1000280, [Single...|1000280|
|{1000310, [Multi-...|1000310|
|{1000360, [Multi-...|1000360|
|{1000370, [Single...|1000370|
|{1000380, [Single...|1000380|
|{1000410, [Single...|1000410|
|{1000470, [Single...|1000470|
|{1000480, [Single...|1000480|
|{1000500, [Multi-...|1000500|
|{1000510, [], 0, ...|1000510|
|{1000540, [Multi-...|1000540|
+--------------------+-------+
only showing top 20 rows



In [0]:
# Nous pouvons voir qu’il s’agit d’une structure imbriquée donc nous devons aplatir la colonne "data"
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

I-Analysis at the "macro" level

In [0]:
# Nous ne pouvons conserver que la colonne 'data' car les id et appid sont dupliqués dans notre dataframe
df_data = df.select("data.*")
df_data.show()

+-------+--------------------+-----+----------------------------+--------+--------------------+--------------------+------------+--------------------+------------------------------------+--------+--------------------+--------------------+--------+-----+----------------------------+------------+------------+-------------------------------------+--------------------+----+--------------------+
|  appid|          categories|  ccu|                   developer|discount|               genre|        header_image|initialprice|           languages|                                name|negative|              owners|           platforms|positive|price|                   publisher|release_date|required_age|                    short_description|                tags|type|             website|
+-------+--------------------+-----+----------------------------+--------+--------------------+--------------------+------------+--------------------+------------------------------------+--------+----------------

In [0]:
# avec cette méthode de pandas, on peut afficher toutes les colonnes de dataset (au lieu d'avoir ... entre les colonnes)
pd.options.display.max_columns = None

In [0]:
df_data.toPandas().head(3)


,appid,categories,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,release_date,required_age,short_description,tags,type,website
0,10,"[Multi-player, Valve Anti-Cheat enabled, Onlin...",13990,Valve,0,Action,https://cdn.akamai.steamstatic.com/steam/apps/...,999,"English, French, German, Italian, Spanish - Sp...",Counter-Strike,5199,"10,000,000 .. 20,000,000","{'linux': True, 'mac': True, 'windows': True}",201215,999,Valve,2000/11/1,0,Play the world's number 1 online action game. ...,"{'1980s': 266.0, '1990's': 1191.0, '2.5D': Non...",game,
1,1000000,"[Single-player, Partial Controller Support, St...",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",https://cdn.akamai.steamstatic.com/steam/apps/...,999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000","{'linux': False, 'mac': False, 'windows': True}",27,999,PsychoFlux Entertainment,2021/05/14,0,ASCENXION is a 2D shoot 'em up game where you ...,"{'1980s': None, '1990's': None, '2.5D': None, ...",game,
2,1000010,"[Single-player, Partial Controller Support, St...",99,NEXT Studios,70,"Adventure, Indie, RPG, Strategy",https://cdn.akamai.steamstatic.com/steam/apps/...,1999,"Simplified Chinese, English, Japanese, Traditi...",Crown Trick,646,"200,000 .. 500,000","{'linux': False, 'mac': False, 'windows': True}",4032,599,"Team17, NEXT Studios",2020/10/16,0,"Enter a labyrinth that moves as you move, wher...","{'1980s': None, '1990's': None, '2.5D': None, ...",game,


In [0]:
dfk_data = df_data.to_koalas()

In [0]:
dfk_data.head(3)

,appid,categories,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,release_date,required_age,short_description,tags,type,website
0,10,"[Multi-player, Valve Anti-Cheat enabled, Onlin...",13990,Valve,0,Action,https://cdn.akamai.steamstatic.com/steam/apps/...,999,"English, French, German, Italian, Spanish - Sp...",Counter-Strike,5199,"10,000,000 .. 20,000,000","{'linux': True, 'mac': True, 'windows': True}",201215,999,Valve,2000/11/1,0,Play the world's number 1 online action game. ...,"{'1980s': 266.0, '1990's': 1191.0, '2.5D': Non...",game,
1,1000000,"[Single-player, Partial Controller Support, St...",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",https://cdn.akamai.steamstatic.com/steam/apps/...,999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000","{'linux': False, 'mac': False, 'windows': True}",27,999,PsychoFlux Entertainment,2021/05/14,0,ASCENXION is a 2D shoot 'em up game where you ...,"{'1980s': None, '1990's': None, '2.5D': None, ...",game,
2,1000010,"[Single-player, Partial Controller Support, St...",99,NEXT Studios,70,"Adventure, Indie, RPG, Strategy",https://cdn.akamai.steamstatic.com/steam/apps/...,1999,"Simplified Chinese, English, Japanese, Traditi...",Crown Trick,646,"200,000 .. 500,000","{'linux': False, 'mac': False, 'windows': True}",4032,599,"Team17, NEXT Studios",2020/10/16,0,"Enter a labyrinth that moves as you move, wher...","{'1980s': None, '1990's': None, '2.5D': None, ...",game,


In [0]:

value_counts = dfk_data["publisher"].value_counts()
# Afficher la valeur maximale et son index (la valeur unique ayant le plus grand nombre d'occurrences)
max_value = value_counts.max()
max_index = value_counts.idxmax()

print(f"La valeur maximale est {max_value} pour '{max_index}'.")

La valeur maximale est 422 pour 'Big Fish Games'.


In [0]:
best_rated=dfk_data.groupby("name")["positive"].max().sort_values(ascending=False)
best_rated


Out[86]: name
Counter-Strike: Global Offensive                                                   5943345
Dota 2                                                                             1534895
Grand Theft Auto V                                                                 1229265
PUBG: BATTLEGROUNDS                                                                1185361
Terraria                                                                           1014711
Tom Clancy's Rainbow Six Siege                                                      942910
Garry's Mod                                                                         861240
Team Fortress 2                                                                     846407
Rust                                                                                732513
Left 4 Dead 2                                                                       643836
The Witcher 3: Wild Hunt                                                    

In [0]:
dfk_data.release_date.dtypes

Out[87]: dtype('O')

In [0]:
#nous devons transformer release_date en date avec datetime:
#la fonction pd.dataframe ne foctionne pas avec Koals , nous utilisons donc le spark SQL 
df_data = df_data.withColumn("date", F.date_format(F.from_unixtime(F.unix_timestamp(F.col("release_date"), "yyyy/MM/d")), "yyyy/MM/dd"))


In [0]:
df_data =df_data.withColumn("datetime", F.to_date(F.col("date"), "yyyy/MM/dd"))

Out[106]: dtype('O')

In [0]:
dfk_data.release_date.dtypes

Out[10]: dtype('O')

In [0]:
dfk_data = df_data.to_koalas()

In [0]:
#supprimer les deux colonnes ajoutées:
dfk_data=dfk_data.drop(columns=["release_date","date"])

In [0]:
dfk_data.head(3)

,appid,categories,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,required_age,short_description,tags,type,website,datetime
0,10,"[Multi-player, Valve Anti-Cheat enabled, Onlin...",13990,Valve,0,Action,https://cdn.akamai.steamstatic.com/steam/apps/...,999,"English, French, German, Italian, Spanish - Sp...",Counter-Strike,5199,"10,000,000 .. 20,000,000","{'linux': True, 'mac': True, 'windows': True}",201215,999,Valve,0,Play the world's number 1 online action game. ...,"{'1980s': 266.0, '1990's': 1191.0, '2.5D': Non...",game,,2000-11-01
1,1000000,"[Single-player, Partial Controller Support, St...",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",https://cdn.akamai.steamstatic.com/steam/apps/...,999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000","{'linux': False, 'mac': False, 'windows': True}",27,999,PsychoFlux Entertainment,0,ASCENXION is a 2D shoot 'em up game where you ...,"{'1980s': None, '1990's': None, '2.5D': None, ...",game,,2021-05-14
2,1000010,"[Single-player, Partial Controller Support, St...",99,NEXT Studios,70,"Adventure, Indie, RPG, Strategy",https://cdn.akamai.steamstatic.com/steam/apps/...,1999,"Simplified Chinese, English, Japanese, Traditi...",Crown Trick,646,"200,000 .. 500,000","{'linux': False, 'mac': False, 'windows': True}",4032,599,"Team17, NEXT Studios",0,"Enter a labyrinth that moves as you move, wher...","{'1980s': None, '1990's': None, '2.5D': None, ...",game,,2020-10-16


In [0]:
display(dfk_data.datetime.dt.year.value_counts())

2021    8805
2020    8287
2018    7663
2022    7451
2019    6949
2017    6006
2016    4176
2015    2566
2014    1550
2013     469
2012     344
2009     309
2010     281
2011     267
2008     159
2007      98
2006      61
2004       6
2005       6
2001       4
1999       3
2003       3
1997       2
2000       2
1998       1
2002       1
Name: datetime, dtype: int64

In [0]:
#l'année 2021 est l'année ou nous avons le plus de sortie de jeu

In [0]:
display(dfk_data.datetime.dt.year.value_counts().sort_index(ascending=False))

2022    7451
2021    8805
2020    8287
2019    6949
2018    7663
2017    6006
2016    4176
2015    2566
2014    1550
2013     469
2012     344
2011     267
2010     281
2009     309
2008     159
2007      98
2006      61
2005       6
2004       6
2003       3
2002       1
2001       4
2000       2
1999       3
1998       1
1997       2
Name: datetime, dtype: int64

In [0]:
 #Au premier vu, oui le covid à impacté n nombre de sorties de jeu, nous constatons une baisse comparant à l'année 2018

In [0]:
dfk_data["initialprice"].value_counts()

Out[122]: 0        7780
499      6518
999      6336
99       5441
199      4285
299      3738
1499     3027
1999     2992
399      2591
699      1964
599      1926
799      1631
2499      936
2999      906
1299      851
1199      850
899       756
1099      460
3999      447
1799      297
1399      272
1599      228
4999      200
5999      194
1699      189
3499      161
1899      150
4499       47
90         34
6999       27
9999       24
500        20
19999      18
100        17
5499       15
7999       14
2199       12
249        12
1500       10
1000       10
300         9
149         9
420         8
200         7
7499        7
6499        7
14999       6
150         6
2299        6
400         6
1249        6
990         6
250         5
2000        5
12999       5
349         5
8999        5
600         5
2399        5
749         4
9499        4
1200        4
129         4
529         4
800         3
666         3
2699        3
2099        3
449         3
2500        3
995       

In [0]:
#Transformer le prix initial pour obtenir le prix réel (on le divise /100)
dfk_data["initialprice"]=(dfk_data["initialprice"].astype(int))/100

In [0]:
##Transformer le prix de vente pour obtenir le prix réel (on le divise /100)
dfk_data["price"]=(dfk_data["price"].astype(int))/100

In [0]:
#Créer une colonne discount pour calculcler la différence entre le prix de vente et le prix initiale
dfk_data["discount"]=dfk_data["initialprice"]-dfk_data["price"]

In [0]:
dfk_data["discount"].value_counts().sort_index()

Out[129]: 0.00     53173
0.10         6
0.15         2
0.18         1
0.20        12
0.22         2
0.25         6
0.30        22
0.30         3
0.36         1
0.37         1
0.40        11
0.40        35
0.40         6
0.42         1
0.45        16
0.45         2
0.45         1
0.48        23
0.49         1
0.50       155
0.55         1
0.60         1
0.60        16
0.60         2
0.63         1
0.65         1
0.66         1
0.68         6
0.69         2
0.70        13
0.70         2
0.71        20
0.75         5
0.80         1
0.80        15
0.80         1
0.90        17
0.96         3
0.99         1
1.00        58
1.02         1
1.05         2
1.08         1
1.10         1
1.10         2
1.14         1
1.18         1
1.20        11
1.20        27
1.22         1
1.25         9
1.26         1
1.30         5
1.30         3
1.32         1
1.35         1
1.38         2
1.40        14
1.40         7
1.42         2
1.48         4
1.50        75
1.50        25
1.60         7
1.60         4


In [0]:
#Conclusion: tous les jeux sont vendus avec une réduction

In [0]:
explode_language=dfk_data.languages.str.split(",").explode()
display(explode_language.value_counts())

/databricks/spark/python/pyspark/sql/pandas/functions.py:386: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(
English                                                                 54646
 German                                                                 13996
 French                                                                 13406
 Russian                                                                12839
 Spanish - Spain                                                        12224
 Simplified Chinese                                                     12213
 Japanese                                                               10170
 Italian                                                                 9297
 Portuguese - Brazil                                                     6739
 

In [0]:
dfk_data.required_age.value_counts()

Out[20]: 0         55030
15          264
18          223
16           38
17           38
12           32
13           26
14           10
10            7
6             4
180           4
3             3
8             3
7             2
35            1
21+           1
5             1
9             1
7+            1
20            1
MA 15+        1
Name: required_age, dtype: int64

In [0]:
required_age_16_18=fk_data[(fk_data.required_age==16)|(fk_data.required_age==16)]